In [ ]:
!pip install rdkit!
!pip install catboost

In [ ]:
! pip install optuna

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import catboost
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split

In [ ]:
df_1400 = pd.read_csv('/content/1400.xlsx - Smile-IC50-CC50.csv')
df_35000 = pd.read_csv('/content/35000.xlsx - Sheet1.csv')
df_more = pd.read_csv('/content/1400.xlsx - Structures-ADMET.csv')
df_fine = pd.read_csv('/content/fine_data.csv')

# Возьмём сначала маленький датасэт и попробуем применить применить к нему градиентный бустинг

In [383]:
df_fine

,Unnamed: 0,Title,IC50,CC50,SI,Molecular weight,Hydrogen bond acceptors,Hydrogen bond donors,Polar SA,SMILES,Pictures
0,0,1007-Ya-213,2.7,500.0,185.185185,195.307,2,1,32.59,OCC\N=C(\[C@]12C)C[C@@H](C1(C)C)CC2,50.0
1,1,1008-Ya-187,9.9,144.0,14.545455,250.431,1,0,15.60,CCN(CC)CC\N=C(\[C@@]12C)C[C@H](C1(C)C)CC2,52.0
2,2,1009-As-106,8.3,500.0,60.240964,222.377,1,0,15.60,CN(C)CC\N=C(\[C@@]12C)C[C@H](C1(C)C)CC2,53.0
3,3,1010-Ya-208,39.4,143.0,3.629442,239.361,2,0,29.54,CN(C)CC(=O)O[C@H]1C[C@H](CC2)C(C)(C)[C@@]12C,54.0
4,4,1011-As-83,25.8,500.0,19.379845,265.466,1,0,12.36,CC[N+](C)(CC)CC\N=C(\[C@]12C)C[C@@H](C1(C)C)CC2,55.0
...,...,...,...,...,...,...,...,...,...,...,...
1252,1252,CHEMBL4241075,15.2,448.5,29.506579,238.330,3,0,39.44,CC(C)CCCCc1c(OC)c(C)c(=O)oc1,NaN
1253,1253,CHEMBL4240549,13.2,448.3,33.962121,224.303,3,0,39.44,CC(C)CCCc1c(OC)c(C)c(=O)oc1,NaN
1254,1254,CHEMBL3186705,24.0,330.5,13.770833,154.254,1,1,20.23,C[C@]12C(C)(C)[C@@H](CC2)C[C@H]1O,NaN
1255,1255,CHEMBL4248255,16.7,492.7,29.502994,224.303,3,1,50.44,CCCCCCCc1c(O)c(C)c(=O)oc1,NaN


In [385]:
df_fine = df_more.replace(',', '.', regex=True)

In [386]:
df_fine.drop(columns = ['Pictures'],axis = 1, inplace=True)
df_fine.drop(columns = ['Unnamed: 56'],axis = 1, inplace=True)
df_fine.drop(columns = ['Unnamed: 55'],axis = 1, inplace=True)

In [387]:
df_fine.dropna(inplace=True)

In [388]:
df_fine.drop(columns='molecule',inplace=True)
SMILES = df_fine['SMILES']
df_fine.drop(columns='SMILES',inplace=True)

In [389]:
IC50 = df_fine['IC50, mmg/ml']
df_fine.drop(columns='IC50, mmg/ml',inplace=True)
CC50_MDCK = df_fine['CC50-MDCK, mmg/ml']
df_fine.drop(columns='CC50-MDCK, mmg/ml',inplace=True)
SI = df_fine['SI']
df_fine.drop(columns='SI',inplace=True)

In [390]:
df_fine

,#stars,#amine,#amidine,#acid,#amide,#rotor,#rtvFG,CNS,mol_MW,dipole,...,SAfluorine,SAamideO,PSA,#NandO,RuleOfFive,#ringatoms,#in34,#in56,#noncon,#nonHatm
0,2.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,195.3,1.3,...,0.00,0.00,35.25,2.0,0.0,0.0,0.0,0.0,0.0,14.0
1,2.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,195.3,1.3,...,0.00,0.00,35.25,2.0,0.0,0.0,0.0,0.0,0.0,14.0
2,2.0,1.0,0.0,0.0,0.0,12.0,0.0,1.0,250.4,2.1,...,0.00,0.00,17.98,2.0,0.0,0.0,0.0,0.0,0.0,18.0
3,3.0,1.0,0.0,0.0,0.0,10.0,0.0,1.0,222.4,2.2,...,0.00,0.00,18.82,2.0,0.0,0.0,0.0,0.0,0.0,16.0
4,1.0,1.0,0.0,0.0,0.0,9.0,1.0,1.0,239.4,5.6,...,0.00,0.00,36.04,3.0,0.0,0.0,0.0,0.0,0.0,17.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1463,1.0,0.0,0.0,0.0,0.0,7.0,1.0,0.0,238.3,5.9,...,0.00,45.62,3.00,0.0,0.0,6.0,0.0,6.0,0.0,17.0
1464,0.0,0.0,0.0,2.0,0.0,7.0,0.0,-2.0,427.4,4.6,...,0.00,134.92,7.00,0.0,1.0,22.0,0.0,22.0,0.0,32.0
1465,0.0,0.0,0.0,2.0,0.0,7.0,0.0,-2.0,427.4,2.6,...,0.00,132.58,7.00,0.0,1.0,22.0,0.0,22.0,0.0,32.0
1466,0.0,0.0,0.0,2.0,0.0,4.0,0.0,-2.0,320.3,3.8,...,0.00,100.97,5.00,0.0,1.0,16.0,0.0,16.0,0.0,24.0


In [391]:
X = df_fine
y = IC50

In [392]:
y

0        2.7
1        0.7
2        9.9
3        8.3
4       39.4
        ... 
1463     7.3
1464     3.8
1465     0.3
1466     0.7
1467     0.1
Name: IC50, mmg/ml, Length: 1394, dtype: object

In [393]:
X

,#stars,#amine,#amidine,#acid,#amide,#rotor,#rtvFG,CNS,mol_MW,dipole,...,SAfluorine,SAamideO,PSA,#NandO,RuleOfFive,#ringatoms,#in34,#in56,#noncon,#nonHatm
0,2.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,195.3,1.3,...,0.00,0.00,35.25,2.0,0.0,0.0,0.0,0.0,0.0,14.0
1,2.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,195.3,1.3,...,0.00,0.00,35.25,2.0,0.0,0.0,0.0,0.0,0.0,14.0
2,2.0,1.0,0.0,0.0,0.0,12.0,0.0,1.0,250.4,2.1,...,0.00,0.00,17.98,2.0,0.0,0.0,0.0,0.0,0.0,18.0
3,3.0,1.0,0.0,0.0,0.0,10.0,0.0,1.0,222.4,2.2,...,0.00,0.00,18.82,2.0,0.0,0.0,0.0,0.0,0.0,16.0
4,1.0,1.0,0.0,0.0,0.0,9.0,1.0,1.0,239.4,5.6,...,0.00,0.00,36.04,3.0,0.0,0.0,0.0,0.0,0.0,17.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1463,1.0,0.0,0.0,0.0,0.0,7.0,1.0,0.0,238.3,5.9,...,0.00,45.62,3.00,0.0,0.0,6.0,0.0,6.0,0.0,17.0
1464,0.0,0.0,0.0,2.0,0.0,7.0,0.0,-2.0,427.4,4.6,...,0.00,134.92,7.00,0.0,1.0,22.0,0.0,22.0,0.0,32.0
1465,0.0,0.0,0.0,2.0,0.0,7.0,0.0,-2.0,427.4,2.6,...,0.00,132.58,7.00,0.0,1.0,22.0,0.0,22.0,0.0,32.0
1466,0.0,0.0,0.0,2.0,0.0,4.0,0.0,-2.0,320.3,3.8,...,0.00,100.97,5.00,0.0,1.0,16.0,0.0,16.0,0.0,24.0


In [394]:
import optuna
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
import catboost as cb
def objective(trial):
    train_x, valid_x, train_y, valid_y = train_test_split(X, y, test_size=0.2)

    param = {
        "objective": 'RMSE',
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.01, 0.1),
        "boosting_type": trial.suggest_categorical("boosting_type", ["Ordered", "Plain"]),
        "bootstrap_type": trial.suggest_categorical(
            "bootstrap_type", ["Bayesian", "Bernoulli"]
        ),
        "used_ram_limit": "10gb",
        'max_depth': trial.suggest_int('max_depth', 3, 8),
        'learning_rate': trial.suggest_categorical('learning_rate', [0.005, 0.02, 0.05, 0.08, 0.1]),
        'n_estimators': trial.suggest_int('n_estimators', 10, 600),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 20, 100),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 0.0001, 1.0, log=True),
        'random_seed': 42,
        'task_type': 'CPU',
        'eval_metric': 'RMSE',
    }

    if param["bootstrap_type"] == "Bayesian":
        param["bagging_temperature"] = trial.suggest_float("bagging_temperature", 0, 10)
    elif param["bootstrap_type"] == "Bernoulli":
        param["subsample"] = trial.suggest_float("subsample", 0.1, 1)

    gbm = cb.CatBoostRegressor(**param)

    gbm.fit(train_x, train_y, eval_set=[(valid_x, valid_y)], verbose=0, early_stopping_rounds=100)

    preds = gbm.predict(valid_x)
    pred_labels = np.rint(preds)
    mse = mean_squared_error(valid_y, pred_labels)
    return mse

In [395]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100, timeout=600, n_jobs=-1)

[I 2023-10-22 03:27:26,561] A new study created in memory with name: no-name-a7cbc916-9dfd-4c44-8ef1-a91249bc47b2
[I 2023-10-22 03:27:26,989] Trial 1 finished with value: 5829.786543369175 and parameters: {'colsample_bylevel': 0.06011765682747119, 'boosting_type': 'Plain', 'bootstrap_type': 'Bernoulli', 'max_depth': 5, 'learning_rate': 0.08, 'n_estimators': 482, 'min_data_in_leaf': 24, 'l2_leaf_reg': 0.2906723398436161, 'subsample': 0.6535389953050247}. Best is trial 1 with value: 5829.786543369175.
[I 2023-10-22 03:27:27,568] Trial 0 finished with value: 4916.185089605735 and parameters: {'colsample_bylevel': 0.07515416117131167, 'boosting_type': 'Ordered', 'bootstrap_type': 'Bayesian', 'max_depth': 3, 'learning_rate': 0.02, 'n_estimators': 538, 'min_data_in_leaf': 43, 'l2_leaf_reg': 0.012232162360806084, 'bagging_temperature': 2.503281177684199}. Best is trial 0 with value: 4916.185089605735.
[I 2023-10-22 03:27:27,765] Trial 2 finished with value: 5298.4181720430115 and parameters: 

In [406]:
study.best_params

{'colsample_bylevel': 0.08667451180776649,
 'boosting_type': 'Ordered',
 'bootstrap_type': 'Bayesian',
 'max_depth': 6,
 'learning_rate': 0.02,
 'n_estimators': 414,
 'min_data_in_leaf': 33,
 'l2_leaf_reg': 0.12352599369810663,
 'bagging_temperature': 2.2864942359968112}

In [407]:
study.best_value

4112.974301075269

In [408]:
the_best_params = study.best_params
the_best_params["objective"] = 'RMSE'
the_best_params["used_ram_limit"] = "10gb"
the_best_params['random_seed'] = 42
the_best_params['task_type'] = 'CPU'
the_best_params['eval_metric'] = 'RMSE'

In [409]:
train_x, valid_x, train_y, valid_y = train_test_split(X, y, test_size=0.2)

In [410]:
gbm1 = cb.CatBoostRegressor(**the_best_params)

gbm1.fit(train_x, train_y, eval_set=[(valid_x, valid_y)], verbose=0, early_stopping_rounds=100)

preds = gbm1.predict(valid_x)

In [411]:
mse = mean_squared_error(valid_y, preds, squared=False)

In [412]:
mse

74.63944524230956

In [404]:
r2_score

0.24759945864765998

In [414]:
from catboost import Pool, cv
cv_dataset = Pool(X, y)
params = the_best_params
scores = cv(cv_dataset, params, fold_count=5)

Training on fold [0/5]
0:	learn: 97.8545496	test: 123.2909456	best: 123.2909456 (0)	total: 4.91ms	remaining: 2.03s
1:	learn: 97.0877141	test: 122.5213603	best: 122.5213603 (1)	total: 19.4ms	remaining: 3.99s
2:	learn: 96.4886486	test: 121.9160523	best: 121.9160523 (2)	total: 31ms	remaining: 4.25s
3:	learn: 95.9009439	test: 121.2857976	best: 121.2857976 (3)	total: 34ms	remaining: 3.48s
4:	learn: 95.2846707	test: 120.6780057	best: 120.6780057 (4)	total: 48.1ms	remaining: 3.94s
5:	learn: 94.7687443	test: 120.1386027	best: 120.1386027 (5)	total: 57.8ms	remaining: 3.93s
6:	learn: 94.1588714	test: 119.5114291	best: 119.5114291 (6)	total: 65.1ms	remaining: 3.78s
7:	learn: 93.6807580	test: 119.0346456	best: 119.0346456 (7)	total: 73.8ms	remaining: 3.75s
8:	learn: 93.2177318	test: 118.5637070	best: 118.5637070 (8)	total: 86.7ms	remaining: 3.9s
9:	learn: 92.7509907	test: 117.9362048	best: 117.9362048 (9)	total: 94ms	remaining: 3.8s
10:	learn: 92.3343350	test: 117.4958473	best: 117.4958473 (10)	to

Результат не удовлетворительный, пытаемся улучьшить нашу модель



---

# теперь используем те же SMILES что и раньше, но используем наши дискрипторы. теперь будем строить модель на новых признаках

In [415]:
smol = pd.read_csv('/content/small_data.csv')
smol.head()
smol.dropna(inplace=True)
smol = smol.replace(',', '.', regex=True)

In [427]:
smol.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,290,291,292,293,294,295,296,297,298,299
0,0,-0.210848,-1.585430,-0.118545,-2.438784,1.577855,-1.791306,-5.109906,3.512098,-0.191232,...,0.395349,5.072766,1.145650,-0.474437,-4.497627,-5.413200,-5.711426,-2.469884,-5.438836,1.620259
1,1,-0.210848,-1.585430,-0.118545,-2.438784,1.577855,-1.791306,-5.109906,3.512098,-0.191232,...,0.395349,5.072766,1.145650,-0.474437,-4.497627,-5.413200,-5.711426,-2.469884,-5.438836,1.620259
2,2,-1.229343,-1.249687,-0.375884,-3.223105,3.549318,-1.426894,-6.608445,3.539260,0.982967,...,1.663534,5.261795,4.314438,-0.115328,-5.096564,-5.632138,-6.515070,-3.512572,-6.707908,-0.930133
3,3,-0.335091,-1.052644,0.148067,-3.004084,2.315342,-1.410794,-4.672865,4.509478,0.801336,...,1.250266,5.929393,2.094081,0.293433,-4.677534,-5.573073,-6.414720,-3.094423,-7.023509,0.889786
4,4,-1.237990,-2.146065,-2.041042,-3.261227,0.628436,-3.924288,-6.218499,3.734038,1.658088,...,0.559931,8.525725,2.371636,1.024816,-6.790434,-6.704412,-6.563089,-4.627900,-8.825220,0.141596


In [416]:
IC50_smol = smol['IC50']
smol.drop(columns='IC50',inplace=True)
CC50_smol = smol['CC50']
smol.drop(columns='CC50',inplace=True)
SI_smol = smol['SI']
smol.drop(columns='SI',inplace=True)

In [417]:
X_smol = smol
y_smol = IC50_smol

In [418]:
def objective(trial):
    train_x, valid_x, train_y, valid_y = train_test_split(X_smol, y_smol, test_size=0.2)

    param = {
        "objective": 'RMSE',
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.01, 0.1),
        "boosting_type": trial.suggest_categorical("boosting_type", ["Ordered", "Plain"]),
        "bootstrap_type": trial.suggest_categorical(
            "bootstrap_type", ["Bayesian", "Bernoulli"]
        ),
        "used_ram_limit": "10gb",
        'max_depth': trial.suggest_int('max_depth', 3, 8),
        'learning_rate': trial.suggest_categorical('learning_rate', [0.005, 0.02, 0.05, 0.08, 0.1]),
        'n_estimators': trial.suggest_int('n_estimators', 10, 600),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 20, 100),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 0.0001, 1.0, log=True),
        'random_seed': 42,
        'task_type': 'CPU',
        'eval_metric': 'RMSE',
    }

    if param["bootstrap_type"] == "Bayesian":
        param["bagging_temperature"] = trial.suggest_float("bagging_temperature", 0, 10)
    elif param["bootstrap_type"] == "Bernoulli":
        param["subsample"] = trial.suggest_float("subsample", 0.1, 1)

    gbm = cb.CatBoostRegressor(**param)

    gbm.fit(train_x, train_y, eval_set=[(valid_x, valid_y)], verbose=0, early_stopping_rounds=100)

    preds = gbm.predict(valid_x)
    pred_labels = np.rint(preds)
    mse = mean_squared_error(valid_y, pred_labels)
    return mse

In [419]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100, timeout=600, n_jobs=-1)

[I 2023-10-22 03:32:58,581] A new study created in memory with name: no-name-9aa706a9-6c48-4db8-b06f-3f81d59d9e4d
[I 2023-10-22 03:32:59,118] Trial 0 finished with value: 164967.27799307957 and parameters: {'colsample_bylevel': 0.03966262860827887, 'boosting_type': 'Plain', 'bootstrap_type': 'Bernoulli', 'max_depth': 3, 'learning_rate': 0.005, 'n_estimators': 320, 'min_data_in_leaf': 70, 'l2_leaf_reg': 0.000624208341776489, 'subsample': 0.22651261005215867}. Best is trial 0 with value: 164967.27799307957.
[I 2023-10-22 03:32:59,240] Trial 1 finished with value: 139620.61536332182 and parameters: {'colsample_bylevel': 0.01809775830032867, 'boosting_type': 'Plain', 'bootstrap_type': 'Bernoulli', 'max_depth': 5, 'learning_rate': 0.08, 'n_estimators': 432, 'min_data_in_leaf': 34, 'l2_leaf_reg': 0.0006006821980863271, 'subsample': 0.8155898245436272}. Best is trial 1 with value: 139620.61536332182.
[I 2023-10-22 03:33:09,403] Trial 3 finished with value: 49689.68048442906 and parameters: {'

In [426]:
study.best_value

17620.28186851211

# Стало только хуже(потому что ножна дальнейшая обработка признаков). предположим, что нам не хватило данный для обучения

# Попробуем теперь большей датасэт. Кажется, что данных должно хватить

In [437]:
big = pd.read_csv('/content/BiG_data (1).csv')
big.head
big = big.replace(',', '.', regex=True)

In [438]:
IC50_big = big['IC50']
big.drop(columns='IC50',inplace=True)
big = big.replace(',', '.', regex=True)

In [439]:
def objective(trial):
    train_x, valid_x, train_y, valid_y = train_test_split(big, IC50_big, test_size=0.2)

    param = {
        "objective": 'RMSE',
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.01, 0.1),
        "boosting_type": trial.suggest_categorical("boosting_type", ["Ordered", "Plain"]),
        "bootstrap_type": trial.suggest_categorical(
            "bootstrap_type", ["Bayesian", "Bernoulli"]
        ),
        "used_ram_limit": "10gb",
        'max_depth': trial.suggest_int('max_depth', 3, 8),
        'learning_rate': trial.suggest_categorical('learning_rate', [0.005, 0.02, 0.05, 0.08, 0.1]),
        'n_estimators': trial.suggest_int('n_estimators', 10, 600),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 20, 100),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 0.0001, 1.0, log=True),
        'random_seed': 42,
        'task_type': 'CPU',
        'eval_metric': 'RMSE',
    }

    if param["bootstrap_type"] == "Bayesian":
        param["bagging_temperature"] = trial.suggest_float("bagging_temperature", 0, 10)
    elif param["bootstrap_type"] == "Bernoulli":
        param["subsample"] = trial.suggest_float("subsample", 0.1, 1)

    gbm = cb.CatBoostRegressor(**param)

    gbm.fit(train_x, train_y, eval_set=[(valid_x, valid_y)], verbose=0, early_stopping_rounds=100)

    preds = gbm.predict(valid_x)
    pred_labels = np.rint(preds)
    mse = mean_squared_error(valid_y, pred_labels, squared=False)
    return mse

In [441]:
# study = optuna.create_study(direction='minimize')
# study.optimize(objective, n_trials=100, timeout=600, n_jobs=-1)

Поиск гиперпараметров не получится запустить, не хватило мозности железа, поэтому просто посмотрим на одну итерацию

In [443]:
train_x, valid_x, train_y, valid_y = train_test_split(big, IC50_big, test_size=0.2)
gbm1 = cb.CatBoostRegressor()
gbm1.fit(train_x, train_y, eval_set=[(valid_x, valid_y)], verbose=1, early_stopping_rounds=100)
preds = gbm1.predict(valid_x)

Learning rate set to 0.085886
0:	learn: 39.5532932	test: 55.9650240	best: 55.9650240 (0)	total: 115ms	remaining: 1m 54s
1:	learn: 37.8940400	test: 55.0890724	best: 55.0890724 (1)	total: 212ms	remaining: 1m 45s
2:	learn: 36.3089887	test: 54.3327244	best: 54.3327244 (2)	total: 309ms	remaining: 1m 42s
3:	learn: 34.8028811	test: 53.7220197	best: 53.7220197 (3)	total: 420ms	remaining: 1m 44s
4:	learn: 33.3519596	test: 53.2380828	best: 53.2380828 (4)	total: 510ms	remaining: 1m 41s
5:	learn: 32.1039561	test: 52.7721321	best: 52.7721321 (5)	total: 603ms	remaining: 1m 39s
6:	learn: 30.8048830	test: 52.3197010	best: 52.3197010 (6)	total: 703ms	remaining: 1m 39s
7:	learn: 29.6323520	test: 51.8275178	best: 51.8275178 (7)	total: 804ms	remaining: 1m 39s
8:	learn: 28.4773377	test: 51.3102215	best: 51.3102215 (8)	total: 892ms	remaining: 1m 38s
9:	learn: 27.4328839	test: 50.9937820	best: 50.9937820 (9)	total: 990ms	remaining: 1m 37s
10:	learn: 26.4108736	test: 50.7348292	best: 50.7348292 (10)	total: 1.

In [446]:
rmse = mean_squared_error(valid_y, preds, squared=False)

In [448]:
study.best_value

16.58522773443738

In [447]:
rmse

48.29534297575317

Показатели заметно улучшелись



---


# пришло время обработать наши признаки




In [465]:
cleanest = pd.read_csv('/content/cleanest.csv')
cleanest.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,292,293,294,295,296,297,298,299,IC50,CC50
0,0,-0.210848,-1.585430,-0.118545,-2.438784,1.577855,-1.791306,-5.109906,3.512098,-0.191232,...,1.145650,-0.474437,-4.497627,-5.413200,-5.711426,-2.469884,-5.438836,1.620259,2.7,500.0
1,1,-1.229343,-1.249687,-0.375884,-3.223105,3.549318,-1.426894,-6.608445,3.539260,0.982967,...,4.314438,-0.115328,-5.096564,-5.632138,-6.515070,-3.512572,-6.707908,-0.930133,9.9,144.0
2,2,-0.335091,-1.052644,0.148067,-3.004084,2.315342,-1.410794,-4.672865,4.509478,0.801336,...,2.094081,0.293433,-4.677534,-5.573073,-6.414720,-3.094423,-7.023509,0.889786,8.3,500.0
3,3,-1.237990,-2.146065,-2.041042,-3.261227,0.628436,-3.924288,-6.218499,3.734038,1.658088,...,2.371636,1.024816,-6.790434,-6.704411,-6.563089,-4.627900,-8.825220,0.141596,39.4,143.0
4,4,2.471257,-3.521509,-0.947530,-1.953958,3.825321,0.132208,-11.256601,5.903816,0.307614,...,7.020583,-0.015244,-6.716763,-7.325947,-4.700589,-5.875819,-7.191969,0.925454,25.8,500.0


In [466]:
cleanest.drop('Unnamed: 0',axis=1, inplace=True)

In [467]:
cleanest['IC50'] = cleanest['IC50'].astype(float)
cleanest['CC50'] = cleanest['CC50'].astype(float)

In [468]:
# Вычислите коэффициенты корреляции между признаками и таргетом
correlations_IC50 = cleanest.corrwith(cleanest['IC50'])
correlations_CC50 = cleanest.corrwith(cleanest['CC50'])
# Отфильтруйте признаки с корреляцией больше 0.1
selected_features_IC50 = correlations_IC50[abs(correlations_IC50) > 0.1].index
selected_features_CC50 = correlations_CC50[abs(correlations_CC50) > 0.1].index
# Сохраните только отфильтрованные признаки
filtered_on_IC50 = cleanest[selected_features_IC50]
filtered_on_CC50 = cleanest[selected_features_CC50]

In [470]:
filtered_on_IC50

,2,3,4,9,11,12,26,29,36,50,...,262,267,268,270,274,275,287,288,IC50,CC50
0,-0.118545,-2.438784,1.577855,-1.455992,1.178078,0.637878,-4.818856,-3.476864,-3.243760,-1.865031,...,1.229612,1.408774,4.814612,5.220927,1.564373,-6.155665,-0.246678,-0.849338,2.7,500.0
1,-0.375884,-3.223105,3.549318,-2.582242,2.754913,1.099961,-4.925318,-2.420787,-3.481503,-1.605160,...,2.373032,1.884635,6.275490,4.034759,1.259240,-7.385296,1.465993,-0.515839,9.9,144.0
2,0.148067,-3.004084,2.315342,-1.973814,1.001105,0.711065,-5.165671,-2.335523,-3.165414,-0.544247,...,1.625621,2.526728,5.801613,5.817666,2.391051,-7.040460,-0.236938,-1.030301,8.3,500.0
3,-2.041042,-3.261227,0.628436,-0.739417,1.076024,2.363109,-3.900088,-3.394971,-1.663674,-0.710014,...,2.950154,1.867724,4.381358,5.562176,2.886764,-5.325635,-0.582024,-1.764034,39.4,143.0
4,-0.947530,-1.953958,3.825321,-1.004157,2.435546,1.836821,-4.212073,-6.553490,-4.038055,-4.183637,...,4.458224,2.226524,5.227225,4.713666,2.117877,-8.168522,2.269230,0.440006,25.8,500.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1252,-3.183022,1.892246,3.303296,-4.418527,2.407567,-1.025365,-3.912335,-0.439184,-1.567971,2.315386,...,-0.500997,1.715598,3.317747,-0.955469,-0.553807,-3.590722,1.869497,-4.129660,15.2,448.5
1253,-3.158019,1.950933,2.863906,-3.824842,2.223059,-0.855225,-3.210555,-0.364085,-1.241827,2.472914,...,-0.703781,1.885040,2.797065,-0.747467,-0.386286,-3.195427,1.460993,-3.895390,13.2,448.3
1254,-1.906609,-2.242947,-0.735992,-1.211951,0.579427,2.938741,-3.166500,-4.047067,-2.000626,-1.658321,...,1.877066,-0.096834,2.894049,5.485382,1.872068,-5.054853,-0.196409,-0.756007,24.0,330.5
1255,-2.263150,1.014681,2.876218,-3.624545,1.285816,-1.860987,-5.007616,-0.476970,-2.285633,1.838779,...,1.623312,-0.041988,3.483566,-1.904540,-1.357877,-2.712623,2.942112,-2.600398,16.7,492.7


получилт почти в 10 раз меньше признаков

Сначала IC50

In [ ]:
IC50 = filtered_on_IC50['IC50']
filtered_on_IC50.drop(columns='IC50',inplace=True)
filtered_on_IC50.drop(columns='CC50',inplace=True)

In [472]:
from scipy.stats import chi2
from sklearn.covariance import MinCovDet

# Загрузите ваш датасет в массив numpy или DataFrame
X = filtered_on_IC50.values

# Оцените ковариацию данных с помощью Minimum Covariance Determinant (MCD)
mcd = MinCovDet()
mcd.fit(X)

# Вычислите Махаланобисовы расстояния для каждой точки данных
mahal_distances = mcd.mahalanobis(X)

# Определите пороговое значение для выбросов с использованием распределения хи-квадрат
threshold = chi2.ppf(0.99, df=X.shape[1])

# Определите, какие точки считаются выбросами
outliers = mahal_distances > threshold

# Удалите выбросы из вашего датасета
filtered_df = filtered_on_IC50[~outliers]

In [473]:
filtered_on_IC50.shape

(1257, 67)

In [474]:
X = filtered_on_IC50
y = IC50

In [475]:
def objective(trial):
    train_x, valid_x, train_y, valid_y = train_test_split(X, y, test_size=0.2)

    param = {
        "objective": 'RMSE',
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.01, 0.1),
        "boosting_type": trial.suggest_categorical("boosting_type", ["Ordered", "Plain"]),
        "bootstrap_type": trial.suggest_categorical(
            "bootstrap_type", ["Bayesian", "Bernoulli"]
        ),
        "used_ram_limit": "10gb",
        'max_depth': trial.suggest_int('max_depth', 3, 8),
        'learning_rate': trial.suggest_categorical('learning_rate', [0.005, 0.02, 0.05, 0.08, 0.1]),
        'n_estimators': trial.suggest_int('n_estimators', 10, 600),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 20, 100),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 0.0001, 1.0, log=True),
        'random_seed': 42,
        'task_type': 'CPU',
        'eval_metric': 'RMSE',
    }

    if param["bootstrap_type"] == "Bayesian":
        param["bagging_temperature"] = trial.suggest_float("bagging_temperature", 0, 10)
    elif param["bootstrap_type"] == "Bernoulli":
        param["subsample"] = trial.suggest_float("subsample", 0.1, 1)

    gbm = cb.CatBoostRegressor(**param)

    gbm.fit(train_x, train_y, eval_set=[(valid_x, valid_y)], verbose=0, early_stopping_rounds=100)

    preds = gbm.predict(valid_x)
    pred_labels = np.rint(preds)
    mse = mean_squared_error(valid_y, pred_labels)
    return mse

In [476]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100, timeout=600, n_jobs=-1)

[I 2023-10-22 04:08:55,649] A new study created in memory with name: no-name-a6143a3b-6d14-4900-bae9-e2f5d30c17d3
[I 2023-10-22 04:08:55,910] Trial 0 finished with value: 653.4517869751827 and parameters: {'colsample_bylevel': 0.04275011295538062, 'boosting_type': 'Plain', 'bootstrap_type': 'Bayesian', 'max_depth': 4, 'learning_rate': 0.08, 'n_estimators': 158, 'min_data_in_leaf': 87, 'l2_leaf_reg': 0.2190854727380835, 'bagging_temperature': 2.60088472437511}. Best is trial 0 with value: 653.4517869751827.
[I 2023-10-22 04:08:55,948] Trial 1 finished with value: 647.987579287496 and parameters: {'colsample_bylevel': 0.02369774221985829, 'boosting_type': 'Plain', 'bootstrap_type': 'Bayesian', 'max_depth': 3, 'learning_rate': 0.1, 'n_estimators': 372, 'min_data_in_leaf': 66, 'l2_leaf_reg': 0.38765029279405516, 'bagging_temperature': 1.8230166214396948}. Best is trial 1 with value: 647.987579287496.
[I 2023-10-22 04:08:57,096] Trial 2 finished with value: 576.9594852932274 and parameters:

In [490]:
study.best_value ** (0.5)

19.422130830691913

In [478]:
study.best_params

{'colsample_bylevel': 0.07578783182038931,
 'boosting_type': 'Ordered',
 'bootstrap_type': 'Bernoulli',
 'max_depth': 5,
 'learning_rate': 0.05,
 'n_estimators': 421,
 'min_data_in_leaf': 39,
 'l2_leaf_reg': 0.004593910121565292,
 'subsample': 0.37055868059175323}

In [491]:
the_best_params = study.best_params
the_best_params["objective"] = 'RMSE'
the_best_params["used_ram_limit"] = "10gb"
the_best_params['random_seed'] = 42
the_best_params['task_type'] = 'CPU'
the_best_params['eval_metric'] = 'RMSE'

In [492]:
train_x, valid_x, train_y, valid_y = train_test_split(X, y, test_size=0.2)

In [493]:
gbm1 = cb.CatBoostRegressor(**the_best_params)

gbm1.fit(train_x, train_y, eval_set=[(valid_x, valid_y)], verbose=0, early_stopping_rounds=100)

preds = gbm1.predict(valid_x)

In [495]:
rmse = mean_squared_error(valid_y, preds)

In [494]:
rmse

48.29534297575317

У нас получилось опять улучшить качество нашей модели!
